# Цель: сгенерировать текст стилем автора, используя предобученную LLM

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import re

from google.colab import drive
from google.colab import files

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

from warnings import filterwarnings
filterwarnings('ignore')

sns.set_style('darkgrid')

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Используется GPU: {torch.cuda.get_device_name(0)}")

Используется GPU: Tesla T4


## Доп. библиотеки 

In [2]:
!pip install Spire.Doc transformers==4.21.0 tokenizers==0.12.1 -U sacremoses evaluate bert_score datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 51.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 78.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 MB 36.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 27.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Successfully uninstalled transfo

### Очищаем .p65

In [8]:
!find . -name "*.p65" -delete

### Считываем все файлы

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


# Чтение текстов с файлов

In [3]:
import pathlib
books = pathlib.Path("/kaggle/input/bolshakov-data/new_data")
files_to_process = []
for book in books.iterdir():
    files_to_process.append(str(book))

In [80]:
files_to_process[0]

'/kaggle/input/bolshakov-data/new_data/Pril.rtf'

# Просмотр и обработка текстов

In [9]:
from spire.doc import *
from spire.doc.common import *
from transformers import T5ForConditionalGeneration, T5Tokenizer
tokenizer_t5 = T5Tokenizer.from_pretrained("cointegrated/rut5-base-paraphraser")

In [10]:
def split_into_chunks(text, max_tokens=512):
    """
    Разбивает текст на чанки по max_tokens токенов, сохраняя границы предложений.
    """
    sentences = re.split(r'(?<=[.!?]) +', text)  
    
    chunks = []
    current_chunk = []
    current_length = 0

    for sent in sentences:
        sent_tokens = tokenizer_t5.tokenize(sent)
        sent_token_count = len(sent_tokens)

        if sent_token_count > max_tokens:
            for i in range(0, sent_token_count, max_tokens):
                chunk = tokenizer_t5.convert_tokens_to_string(sent_tokens[i:i+max_tokens])
                chunks.append(chunk)
            continue

        if current_length + sent_token_count > max_tokens:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
            current_length = 0

        current_chunk.append(sent)
        current_length += sent_token_count

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

In [77]:
def clean_doc_and_get_chunks(file):
    doc = Document()
    doc.LoadFromFile(file)
    text = doc.GetText()
    text = text.replace('Evaluation Warning: The document was created with Spire.Doc for Python.', '')
    text = text.replace('\t', '')
    clean_text = re.sub(
        r'\d+\r\n|\r\n|\x0c|УДК.*|ББК.*|Издание.*|– М.*|ISBN.*|©.*',
        '\n', 
        text
    )
    final_chunks = []
    paragraphs = [p.strip() for p in clean_text.split('\n') if p.strip()]
    for paragraph in paragraphs:
        if len(paragraph) < 100:
            continue
        chunks = split_into_chunks(paragraph)
        final_chunks.extend(chunks)
    return final_chunks

In [86]:
textes = clean_doc_and_get_chunks(files_to_process[0])

In [88]:
for i, chunk in enumerate(final_chunks):
    if i <= 10:
        print(f"Часть {i+1}, токенов: {len(tokenizer_t5.tokenize(chunk))}")
        print(chunk[:200] + "...\n")

Часть 1, токенов: 375
В функционировании социально-политических систем первостепенную роль играют обратные связи. Более того, любые системы с самоорганизацией способны успешно функционировать лишь при наличии соответствующ...

Часть 2, токенов: 183
Кратко напомним, что такое управление с обратными связями и какие обратные связи существуют. Под управлением с обратной связью понимают такое управление, когда результаты процесса воздействуют на его ...

Часть 3, токенов: 230
Стимулирующие обратные связи делят на положительные и отрицательные в зависимости от того, усиливают или ослабляют они управляемый  процесс. Положительная обратная связь приводит ко все большему откло...

Часть 4, токенов: 127
Общий для всех систем способ их организации прост: на выходе какого-либо процесса (иногда в специальном элементе сравнения) сравниваются заранее запланированный и реально достигнутый результаты. Сигна...

Часть 5, токенов: 188
Обратные связи могут осуществляться как с помощью специализированных э

# Генерирование тестовой выборки с помощью перефразеров



## Функция переплексии - метрика, которая вычисляет схожесть текстов по смыслу, применяя модель Сбера

In [36]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm.auto import tqdm

In [37]:
mname = 'ai-forever/rugpt3small_based_on_gpt2'
gpt_tokenizer = AutoTokenizer.from_pretrained(mname)
gpt_model = AutoModelForCausalLM.from_pretrained(mname)
gpt_model.cuda();

In [38]:
def get_gpt2_ppl_corpus(test_textes, aggregate=True, sep='\n', max_length=1024):
    lls, weights = [], []
    for text in tqdm(test_textes):
        encodings = gpt_tokenizer(f'{sep}{text}{sep}', return_tensors='pt', padding=True, truncation=True, max_length=max_length)
        input_ids = encodings.input_ids.to(gpt_model.device)
        target_ids = input_ids.clone()

        w = max(0, len(input_ids[0]) - 1)
        if w > 0:
            with torch.no_grad():
                output = gpt_model(input_ids, labels=target_ids)
                likelihood = output[0]
                ll = likelihood.item()
        else:
            ll = 0
        lls.append(ll)
        weights.append(w)
    likelihoods, weights = np.array(lls), np.array(weights)
    if aggregate:
        return sum(likelihoods * weights) / sum(weights)
    return likelihoods, weights

------

## BLEU 

In [39]:
from nltk.translate.bleu_score import sentence_bleu

------------

## BERTScore

In [14]:
from evaluate import load
bertscore = load("bertscore")

## Валидация работы предобученной T5 на задаче перефразирования 

In [14]:
model = T5ForConditionalGeneration.from_pretrained("cointegrated/rut5-base-paraphraser").to(device)

In [23]:
len(tokenizer_t5.tokenize(final_chunks[11]))

181

In [38]:
text = final_chunks[11]
input_ids = tokenizer_t5.encode("paraphrase: " + text, return_tensors="pt").to(device)
outputs = model.generate(
    input_ids,
    do_sample=True,           # ← Включить сэмплирование
    temperature=1,          # Случайность (0.5-1.2)
    top_k=50,                 # Ограничить выбор топ-50 токенами
    top_p=0.95,               # Nucleus sampling
    max_length=181+50,
    min_length=181,
    no_repeat_ngram_size=3
)
decoded = tokenizer_t5.decode(outputs[0], skip_special_tokens=True)

In [36]:
decoded

'Таким образом, в результате правильно проведенного стимулирования контактов у людей, занимающихся управлениями, должно формироваться желание действовать не только в интересах личного, но и более широких объединений граждан (например, в акционерном обществе или корпорации), то есть появляется реальная возможность для оптимального согласования интересов своих и общественных организаций. Основная сила связей: отдельно взятое руководство и контролирует посторонние внешние объекты, которые должны работать против серьезных интересов и заинтересованных, иначе говоря. Очередь - значит, было преданное государствам. <b> Относительно темы и представлений на сайте является: <b] <p> <p</p> > <p]> [p>'

In [37]:
final_chunks[11]

'Правильная организация обратных связей должна приводить к модификации вторичных интересов, их коррекции в сторону общественно полезных, а не наоборот, как это происходит зачастую сейчас. Героем нашего времени стал не труженик, ученый, летчик или разведчик, а вор, обманщик, казнокрад и карточный жулик. С введением же правильно организованных стимулирующих обратных связей у человека, участвующего в организации управления, появляется желание действовать не только в личных интересах, но и в интересах более широких объединений граждан (например, в акционерном обществе или корпорации), т.е. появляется реальная возможность оптимального согласования своих и общественных интересов.'

In [31]:
print(f'Значение переплекcии: {get_gpt2_ppl_corpus(decoded)}')

  0%|          | 0/796 [00:00<?, ?it/s]

Значение переплекcии: 6.361718785554211


## Сбор маленького датасета 

In [72]:
def generate_text(file):
    samples = []
    for t in file:
        min_len = len(tokenizer_t5.tokenize(t))
        input_ids = tokenizer_t5.encode("paraphrase: " + t, return_tensors="pt").to(device)
        outputs = model.generate(
            input_ids,
            do_sample=True,           # ← Включить сэмплирование
            temperature=1,          # Случайность (0.5-1.2)
            top_k=50,                 # Ограничить выбор топ-50 токенами
            top_p=0.95,               # Nucleus sampling
            max_length=min_len+50,
            min_length=min_len,
            no_repeat_ngram_size=3
        )
        decoded = tokenizer_t5.decode(outputs[0], skip_special_tokens=True)
        samples.append(''.join(decoded))
    return samples

In [89]:
file1 = clean_doc_and_get_chunks(files_to_process[0])
file2 = clean_doc_and_get_chunks(files_to_process[1])
file3 = clean_doc_and_get_chunks(files_to_process[2])

In [90]:
train_data_first_file = generate_text(file1)
train_data_second_file = generate_text(file2)
train_data_third_file = generate_text(file3)

In [98]:
len(train_data_first_file)

67

In [99]:
finetuning_data = pd.DataFrame({'train': train_data_first_file + train_data_second_file + train_data_third_file, 
                                'target': file1 + file2 + file3
                               }) 

In [100]:
finetuning_data.shape

(324, 2)

In [107]:
finetuning_data.to_csv('small_dataset.csv', sep=',', index=False, encoding='utf-8')

In [46]:
finetuning_data = pd.read_csv('/kaggle/input/bolshakov-small-dataset/small_dataset.csv')

In [47]:
finetuning_data.head()

,train,target
0,Вопрос о деятельности социополитических систем...,В функционировании социально-политических сист...
1,- Подробнее: Существует понятие управления свя...,"Кратко напомним, что такое управление с обратн..."
2,"Например, применение стимулирующей обратной св...",Стимулирующие обратные связи делят на положите...
3,- Общая форма организации для всех систем прос...,Общий для всех систем способ их организации пр...
4,"Для связи, при которой предусмотрена система, ...",Обратные связи могут осуществляться как с помо...


In [43]:
print(f'Средняя переплексия в маленьком датасете: {get_gpt2_ppl_corpus(small_data["train"]):.4f}')

  0%|          | 0/324 [00:00<?, ?it/s]

Средняя переплексия в маленьком датасете: 4.5989


In [44]:
all_metrics = []
for i in range(len(small_data)):
    hypotheses = small_data['train'][i].split()
    references = small_data['target'][i].split()
    all_metrics.append(sentence_bleu([references], hypotheses))
print(f'Средний BLEU в маленьком датасете: {np.mean(all_metrics):.4f}')

Средний BLEU в маленьком датасете: 0.1960


In [45]:
print(f'Средний BERTScore в маленьком датасете: {(np.mean(bertscore.compute(predictions=small_data["train"], references=small_data["target"], lang="ru")["f1"])):.4f}')

Средний BERTScore в маленьком датасете: 0.8155


# Дообучение T5 на маленьком датасете

In [4]:
from datasets import Dataset